In [1]:
# Create a wide and deep neural network, JC recommended creating a class that combines many aspects

In [2]:
!pip install category_encoders
!pip install spacy
!python -m spacy download en_core_web_lg

    100% |████████████████████████████████| 102kB 28.7MB/s a 0:00:01
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 10.4MB 5.0MB/s eta 0:00:01
    100% |████████████████████████████████| 20.0MB 2.5MB/s eta 0:00:01
    100% |████████████████████████████████| 2.2MB 23.6MB/s ta 0:00:01
    100% |████████████████████████████████| 122kB 41.0MB/s ta 0:00:01
    100% |████████████████████████████████| 3.7MB 17.9MB/s ta 0:00:01
    100% |████████████████████████████████| 194kB 42.6MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 32.1MB/s ta 0:00:01
  Found existing installation: numpy 1.14.3
    Uninstalling numpy-1.14.3:
      Successfully uninstalled numpy-1.14.3
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |███████████████████

In [3]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [1]:
import spacy

import numpy as np

import boto3
import pandas as pd
from sagemaker import get_execution_role

import category_encoders as ce

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize.regexp import regexp_tokenize
from nltk.stem import WordNetLemmatizer

In [2]:
# Dataset

role = get_execution_role()
bucket='sagemaker-tagger'
data_key1 = 'update_twelve_three.csv'
data_location1 = 's3://{}/{}'.format(bucket, data_key1)

df = pd.read_csv(data_location1)

In [3]:
df.head()

,From,Message,Subject,Tags,UID,text,tag_list,first_tag,sender_name,sender_email,domain_name,isNoReply
0,<grangepayments@westernunionspeedpay.com>,"Dear AVRAHAM JACOBSOHN, This is to confirm th...",Grange Payment Confirmation,Finance,31780,<grangepayments@westernunionspeedpay.com> Gran...,['Finance'],Finance,NaN,grangepayments@westernunionspeedpay.com,westernunionspeedpay,False
1,Chase <no.reply.alerts@chase.com>,This is an Alert to help manage your account ...,Your Debit Card Transaction,Finance,31779,Chase <no.reply.alerts@chase.com> Your Debit C...,['Finance'],Finance,Chase,no.reply.alerts@chase.com,chase,True
2,Amazon Web Services <no-reply-aws@amazon.com>,Please let us know if we helped resolve your i...,Resolved 6559329691: Limit Increase: SageMaker,Productivity,31738,Amazon Web Services <no-reply-aws@amazon.com> ...,['Productivity'],Productivity,Amazon Web Services,no-reply-aws@amazon.com,amazon,True
3,Lambda Labs <noreply@github.com>,Youve been added to the Labs 18 - Tagger team ...,Bernie Durfee added you to the Lambda Labs tea...,Productivity,31693,Lambda Labs <noreply@github.com> Bernie Durfee...,['Productivity'],Productivity,Lambda Labs,noreply@github.com,github,True
4,Amazon Web Services <no-reply-aws@amazon.com>,"Hello, We haven't heard back from you regard...",Attention required on case 6559329691: Limit I...,Productivity,31684,Amazon Web Services <no-reply-aws@amazon.com> ...,['Productivity'],Productivity,Amazon Web Services,no-reply-aws@amazon.com,amazon,True


In [4]:
# Applying length of messages as feature

df['Text_Length'] = df['Message'].apply(str)
df['Text_Length'] = df['Text_Length'].apply(len)
print(df['Text_Length'].min(), df['Text_Length'].max())

1 3732404


In [5]:
# Function to clean the text prior to tokenizing

def clean_text(text):
    # Perform a few cleaning steps to remove non-alphabetic characters
    
    text = text.replace("\n", " ").replace("\r", " ")
    
    text = text.strip(" ")
    
    punc_list = '!@#$%^&*()+?-_=:.<>[]{}/\~",' + '1234567890'
    t = str.maketrans(dict.fromkeys(punc_list, " "))
    text = text.translate(t)
    
    return text

df['Clean_Message'] = (df['text'].apply(clean_text))

In [6]:
# Applying length of CLEAN messages as feature

df['Clean_Text_Length'] = df['Clean_Message'].apply(len)
print(df['Clean_Text_Length'].min(), df['Clean_Text_Length'].max())

29 3732441


In [7]:
# Most lengths are below 5000 even with a max of 370k length

clean_text_length = df['Clean_Text_Length'] < 5000
clean_text_length.value_counts()

True     10532
False      666
Name: Clean_Text_Length, dtype: int64

In [8]:
X = df['Clean_Message']
y = df['first_tag']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8958,), (2240,), (8958,), (2240,))

In [9]:
lemmatizer = WordNetLemmatizer()

def regnltk_tokenize(text):
    text = clean_text(text)
    words = regexp_tokenize(text, pattern = '\s+', gaps = True)
    return [lemmatizer.lemmatize(word.lower()) for word in words if (len(word) >= 3)] 

In [10]:
X_train.iloc[3]

" Two Plus Two Poker Forums   forum master twoplustwo com   Merge Network  Carbon  Sportsbook  PlayersOnly   Deposit  Withdrawal Thread  update Dear noreallyimfine   You are subscribed to the thread  Merge Network  Carbon  Sportsbook  PlayersOnly   Deposit  Withdrawal Thread  by kahntrutahn  there have been   post s  to this thread  the last poster was highinaroom  http   forumserver twoplustwo com showthread php t          These following posts were made to the thread               Re  Merge Network  Carbon  Sportsbook  PlayersOnly   Deposit  Withdrawal Thread http   forumserver twoplustwo com showthread php p          post         Posted by  highinaroom On                   AM  So I had Circle to move funds from Blockchain to my bank account but Circle stopped doing this  What's the alternative to Circle now               Re  Merge Network  Carbon  Sportsbook  PlayersOnly   Deposit  Withdrawal Thread http   forumserver twoplustwo com showthread php p          post         Posted by  

In [14]:
vect = TfidfVectorizer(tokenizer=regnltk_tokenize, min_df=0.03, max_df=0.98, stop_words='english')
X_train = vect.fit_transform(X_train)
X_test = vect.transform(X_test)

encoder = ce.OrdinalEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [15]:
nlp = spacy.load('en_core_web_lg')

## Starting LSTM set-up with Keras

In [16]:
!pip install keras

    100% |████████████████████████████████| 378kB 25.3MB/s ta 0:00:01
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [11]:
import seaborn as sns
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
from bs4 import BeautifulSoup
import plotly.graph_objs as go

Using TensorFlow backend.
/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 =

In [12]:
# Starting tokenizing from keras preprocessing

tokenizer = Tokenizer(num_words=5000, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['Clean_Message'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 443652 unique tokens.


In [13]:
X_LSTM = tokenizer.texts_to_sequences(df['Clean_Message'].values)
X_LSTM = pad_sequences(X_LSTM, maxlen=5000)
print('Shape of data tensor:', X_LSTM.shape)

Shape of data tensor: (11198, 5000)


In [14]:
Y_LSTM = pd.get_dummies(df['first_tag']).values
print('Shape of label tensor:', Y_LSTM.shape)

Shape of label tensor: (11198, 8)


In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(X_LSTM,Y_LSTM, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(10078, 5000) (10078, 8)
(1120, 5000) (1120, 8)


In [16]:
model = Sequential()
model.add(Embedding(5000, 100, input_length=X_LSTM.shape[1]))
model.add(SpatialDropout1D(0.15))
model.add(LSTM(100, dropout=0.15, recurrent_dropout=0.15))
model.add(Dense(8, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 5000, 100)         500000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 5000, 100)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 808       
Total params: 581,208
Trainable params: 581,208
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
epochs = 2
batch_size = 8

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])